In [46]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error

## Task 3

In [47]:
# Load data
dfTrainData = pd.read_csv("./../TrainData.csv")
dfTrainData

,TIMESTAMP,POWER,U10,V10,WS10,U100,V100,WS100
0,20120101 1:00,0.273678,0.534894,-3.660243,3.699120,0.765355,-4.486657,4.551468
1,20120101 2:00,0.086796,0.330813,-2.676430,2.696797,0.469875,-3.213752,3.247920
2,20120101 3:00,0.006811,-0.065839,-2.029072,2.030140,-0.020893,-2.418119,2.418209
3,20120101 4:00,0.018646,-0.419549,-1.799090,1.847362,-0.444867,-2.149654,2.195203
4,20120101 5:00,0.034812,-0.754224,-1.661526,1.824698,-0.838274,-1.991829,2.161038
...,...,...,...,...,...,...,...,...
16075,20131031 20:00,0.101254,-1.459917,-1.778960,2.301316,-1.974368,-4.936280,5.316483
16076,20131031 21:00,0.105047,-1.293326,-2.350447,2.682777,-0.781394,-4.903565,4.965433
16077,20131031 22:00,0.145079,-0.308284,-2.441885,2.461269,0.257905,-4.027432,4.035681
16078,20131031 23:00,0.180933,1.148765,-2.789011,3.016330,1.628004,-3.520358,3.878572


In [48]:
# Limit use of wind speed at 10m above ground level

# TrainData
dfTrainData = dfTrainData.drop(columns=['U100', 'V100', 'WS100'])

# WeatherForecastInput
dfWFI = pd.read_csv("./../WeatherForecastInput.csv")
dfWFI = dfWFI.drop(columns=['U100', 'V100', 'WS100'])
dfWFI

,TIMESTAMP,U10,V10,WS10
0,20131101 1:00,3.462251,-1.066374,3.622753
1,20131101 2:00,3.651341,-0.337717,3.666925
2,20131101 3:00,3.657829,0.328049,3.672510
3,20131101 4:00,3.551698,0.735371,3.627028
4,20131101 5:00,3.586921,1.050811,3.737673
...,...,...,...,...
715,20131130 20:00,-2.527419,-6.623859,7.089666
716,20131130 21:00,-2.529020,-7.423845,7.842794
717,20131130 22:00,-3.105103,-6.653005,7.341943
718,20131130 23:00,-2.286417,-6.165074,6.575396


In [49]:
# Make wind power production forecasting when we only have wind power generation data; and we do not have other data

# The following columns should be removed: U10, V10, WS10, U100, V100, WS100
# In the new training data file, we only have two columns: TIMESTAMP and POWER, which is called as time-series data
dfTrainData = dfTrainData.drop(columns=['U10', 'V10', 'WS10'])

In [60]:
# Convert TIMESTAMP (string) to float
timestamp = []
for index, row in dfTrainData.iterrows():
    temp_tp = datetime.strptime(row['TIMESTAMP'], '%Y%m%d %H:%M')
    timestamp.append(datetime.strftime(temp_tp, '%m-%d'))
#dfTrainData['TIMESTAMP'] = timestamp
print(timestamp)

['01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-01', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-02', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-03', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-04', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', '01-05', 

In [61]:
""" Linear Regression """
# Separate features and target variable
X_train = timestamp
y_train = dfTrainData["POWER"].values

# Train the linear regression model
LR_model = LinearRegression()
LR_model.fit(X_train, y_train)

# Predict the wind power production for the whole month 11.2013 based on the MLR model and weather data in WeatherForecastInput.csv
X_test = dfWFI[['TIMESTAMP']].values

# Prediction
y_pred = LR_model.predict(X_test)

# Evaluate the model
dfSolution = pd.read_csv("./../Solution.csv")
print('Root mean squared error (RMSE):', np.sqrt(mean_squared_error(dfSolution['POWER'], y_pred)))

# save to ForecastTemplate1-LR.csv

ValueError: Expected 2D array, got 1D array instead:
array=['01-01' '01-01' '01-01' ... '10-31' '10-31' '11-01'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
""" Support Vector Regression """


In [ ]:
""" Artificial Neural Network (ANN) """



In [20]:
dfSolution = pd.read_csv("./../Solution.csv")
dfSolution
#print('Root mean squared error (RMSE):', np.sqrt(mean_squared_error(dfSolution['POWER'], y_test)))

,TIMESTAMP,POWER
0,20131101 1:00,0.167215
1,20131101 2:00,0.063998
2,20131101 3:00,0.039035
3,20131101 4:00,0.036232
4,20131101 5:00,0.064888
...,...,...
715,20131130 20:00,0.963328
716,20131130 21:00,0.948834
717,20131130 22:00,0.925991
718,20131130 23:00,0.773840
